# Import

In [1]:
import numpy as np

In [2]:
def MSE(y_pred : np.array, y : np.array):
    return np.mean((y_pred - y) ** 2)

In [3]:
class LinearRegressionImplement:
    def __init__(self, learning_rate=0.01, epochs=100, l1_reg=0.0, l2_reg=0.0, tol=1e-4):
        """
        Initialize the Linear Regression model.

        Parameters:
            learning_rate (float): The learning rate for gradient descent.
            epochs (int): Number of training iterations.
            l1_reg (float): L1 regularization coefficient (lambda1).
            l2_reg (float): L2 regularization coefficient (lambda2).
            tol (float): Tolerance for early stopping. Training stops if the absolute
                         improvement in loss between epochs is less than tol.
        """
        self.lr = learning_rate       # Learning rate for gradient descent
        self.epochs = epochs          # Maximum number of training epochs
        self.l1_reg = l1_reg          # L1 regularization parameter (λ₁)
        self.l2_reg = l2_reg          # L2 regularization parameter (λ₂)
        self.tol = tol                # Tolerance for early stopping based on loss improvement
        self.weights = None           # Model parameters (weights + bias)
        self.loss_history = []        # List to track the MSE loss at each epoch

    def fit(self, X, y):
        """
        Fit the linear regression model to the training data using Stochastic Gradient Descent.

        Parameters:
            X (np.array): Feature matrix.
            y (np.array): Target vector.

        Returns:
            self: The fitted model.
        """
        # Add a bias term (column of ones) to the input feature matrix X
        X_b = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
        
        # Initialize weights randomly (including bias)
        n_features = X_b.shape[1]
        self.weights = np.random.randn(n_features)
        
        # Training loop over epochs
        for epoch in range(self.epochs):
            # Shuffle the data to avoid order bias
            shuffled_indices = np.random.permutation(X_b.shape[0])
            X_shuffled = X_b[shuffled_indices]
            y_shuffled = y[shuffled_indices]
            
            # Update weights for each data point (Stochastic Gradient Descent)
            for i in range(X_shuffled.shape[0]):
                xi = X_shuffled[i]
                yi = y_shuffled[i]
                
                # Compute prediction for the current sample and the error
                y_pred = np.dot(xi, self.weights)
                error = y_pred - yi
                
                # Compute gradient from the MSE loss: (y_pred - y) * xi
                grad = error * xi
                
                # Create a mask to avoid regularizing the bias term (first element)
                reg_mask = np.concatenate(([0], np.ones(len(self.weights) - 1)))
                
                # Compute the gradient from L2 regularization: λ₂ * weight
                grad_l2 = self.l2_reg * self.weights * reg_mask
                
                # Compute the subgradient from L1 regularization: λ₁ * sign(weight)
                grad_l1 = self.l1_reg * np.sign(self.weights) * reg_mask
                
                # Total gradient is the sum of the loss gradient and regularization gradients
                total_grad = grad + grad_l2 + grad_l1
                
                # Update the weights using the gradient descent rule
                self.weights -= self.lr * total_grad
            
            # Calculate the loss over the entire dataset for monitoring after each epoch
            y_pred_all = np.dot(X_b, self.weights)
            mse = MSE(y_pred_all, y)
            self.loss_history.append(mse)
            
            # Optionally print the loss every 10 epochs (or adjust as desired)
            if epoch % 5 == 0:
                print(f"Epoch {epoch}: Loss {mse}")
            
            # Early stopping: stop if the absolute improvement is below the tolerance
            if epoch > 0:
                loss_improvement = np.abs(self.loss_history[-2] - mse)
                if loss_improvement < self.tol:
                    print(f"Early stopping at epoch {epoch}. Loss improvement {loss_improvement} is below tolerance {self.tol}.")
                    break
            
        return self
    
    def predict(self, X):
        """
        Make predictions using the trained linear regression model.

        Parameters:
            X (np.array): Feature matrix.

        Returns:
            np.array: Predicted values.
        """
        # Add a bias term (column of ones) to the input feature matrix X
        X_b = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
        return np.dot(X_b, self.weights)

# Test 

## 1 feature

In [4]:
np.random.seed(42)
(n, m) = (100, 1)
X = np.random.rand(n, m)  # Shape (100, 3)
w = [2]
bias = 3
y = bias + X@w + np.random.randn(n)  # y = 3 + 2X1 + noise

In [5]:
# Initialize and fit the model
model = LinearRegressionImplement(learning_rate=0.01, epochs=50, l1_reg=0.001, l2_reg=0.002)
model.fit(X, y)

# Final weights (bias term is the first element)
print("Weights: [bias, coefficients]:", model.weights)

Epoch 0: Loss 1.8574816925743982
Epoch 5: Loss 0.8613410625071986
Epoch 10: Loss 0.8321739055416505
Epoch 15: Loss 0.8179377750676542
Epoch 20: Loss 0.8116299100442721
Epoch 25: Loss 0.8096940268248987
Early stopping at epoch 26. Loss improvement 9.444656018553488e-05 is below tolerance 0.0001.
Weights: [bias, coefficients]: [3.18006317 1.68976879]


## 3 features

In [6]:
np.random.seed(42)
(n, m) = (100, 3)
X = 2 * np.random.rand(n, m)  # Shape (100, 3)
w = [2, 1.5, -0.5]
bias = 3
y = bias + X@w + np.random.randn(n)  # y = 3 + 2X1 + 1.5X2 - 0.5X3 + noise

In [7]:
# Initialize and fit the model
model = LinearRegressionImplement(learning_rate=0.01, epochs=50, l1_reg=0.001, l2_reg=0.002)
model.fit(X, y)

# Final weights (bias term is the first element)
print("Weights: [bias, coefficients]:", model.weights)

Epoch 0: Loss 1.8970453822262159
Epoch 5: Loss 1.0214239770044546
Epoch 10: Loss 0.9498859630879611
Epoch 15: Loss 0.9344863645894534
Epoch 20: Loss 0.9287210939837265
Epoch 25: Loss 0.945166321569708
Epoch 30: Loss 0.9249811330352901
Epoch 35: Loss 0.925795321017349
Early stopping at epoch 38. Loss improvement 5.2096268449264294e-05 is below tolerance 0.0001.
Weights: [bias, coefficients]: [ 2.67869561  2.13328726  1.43005693 -0.19545391]
